Este artigo científico aborda o crescente problema do spam por SMS, destacando a escassez de conjuntos de dados públicos para pesquisa e avaliação de filtros. Para superar essa limitação, os autores apresentam uma nova e extensa coleção de mensagens SMS reais e não codificadas, considerada a maior disponível até então. O estudo também compara o desempenho de diversos métodos de aprendizado de máquina na filtragem de spam, revelando que o Support Vector Machine (SVM) obteve os melhores resultados, estabelecendo uma importante linha de base para futuras comparações. A pesquisa conclui que, apesar dos desafios inerentes à natureza concisa e informal das mensagens SMS, o SVM demonstra ser uma técnica promissora para combater o spam móvel.

# Passo 1: Carregamento e Exploração

In [3]:
# Importando as bibliotecas
import requests
import zipfile
import io
import pandas as pd

In [5]:
# Carregando o dataset 
url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
response = requests.get(url)
response.raise_for_status() # Lança uma exceção se o download falhar

In [7]:
#descompactando os arquivos
with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
    zip_ref.extractall(".") # Extrai todos os arquivos para o diretório atual

In [9]:
df_sms = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['label', 'message'])

In [11]:
df_sms.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [13]:
# Analisando a distribuição ham/spam
print(df_sms['label'].value_counts())

label
ham     4825
spam     747
Name: count, dtype: int64


In [15]:
# Exemplos de cada classe
print("\nExemplo de Ham:")
print(df_sms[df_sms['label'] == 'ham']['message'].iloc[0])
print("\nExemplo de Spam:")
print(df_sms[df_sms['label'] == 'spam']['message'].iloc[0])


Exemplo de Ham:
Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

Exemplo de Spam:
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


# Passo 2: Pré-processamento

In [17]:
import re

def limpar_texto(texto):
    texto = re.sub(r'[^a-zA-Z]', ' ', texto)  # Remove caracteres especiais e números
    texto = texto.lower()  # Converte para minúsculas
    texto = texto.strip()  # Remove espaços em branco extras
    return texto

df_sms['message'] = df_sms['message'].apply(limpar_texto)

# Tokenização (exemplo com split simples)
df_sms['tokens'] = df_sms['message'].apply(lambda x: x.split())

# (Opcional) Testar stopwords e stemming aqui